# Implementation

In [1]:
import string
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import pandas as pd

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\deepi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Reading the Text File

In [2]:
text1=open(r"C:\Users\deepi\Desktop\SEM 2\Text Analytics\10930112.txt","rt", encoding="utf8")
pride=text1.read()

#### Data cleaning function

In [3]:
def cleaning(textfile):
    y=textfile
    ##step 1: converting to lower case
    y=y.lower()
    
    #step 2: replacing apostraphe
    y=y.replace("'", " ")
    
    # step 3: remove_punctuation
    y=y.translate(str.maketrans(string.punctuation,' '*len(string.punctuation)))
    
    #step4: removing numbers
    num="0123456789"
    y=y.translate(str.maketrans(num,' '*len(num)))
    
    #step5: removing empty lines
    cleaned='\n'.join([x for x in y.split("\n") if x.strip()!=''])
    return cleaned

#### Splitting function -  split1 includes the first 5000 lines of the text file, split2 contains the remaining lines

In [4]:

def split_function(text):
    x=text.splitlines()
    count=0
    split1=x[0:5000]
    split2=x[5000:]
    return split1,split2

        
    

#### Mapper function

In [5]:
def mapper(text,queue):
    x=text
    map=list()
    for i in range(len(x)):
        y=x[i].split()       
        for m in y:
            map.append([m,1]) #creating a list with key value pairs
        
    queue.put(map)
 

#### sorting function

In [6]:
def sorting(text1,text2):
    x1=text1+text2
    x1.sort(key=lambda x1:x1[0]) #sorting the keys in alphabetical order
    return x1
    
    

#### Partition function

In [7]:
def partition(text):
    new = {}
    for (key, value) in text:   # Converting the sorted words to dictionary
        if key in new:
            new[key].append(value)   # appending the values of a particular word if a key has many values
        else:
            new[key] = [value]      
            letters1 = ('a','b','c','d','e','f','g','h','i','j','k','l','m')
            res1 ={key:val for key, val in new.items()  
                   if key.startswith(letters1)}         # This will return the words that are starting with letters a to m
            res2 = {key:val for key, val in new.items()  
                   if not key.startswith(letters1)}     # This will return the words that are starting with letters n to z
    return(res1),(res2)

#### Reducer function

In [8]:
def reducer(a,queue):    
    a1 = dict(zip(a.keys(), [sum(item) for item in a.values()])) # Summing all the values with same key from the dictionary
    queue.put(a1)

#### Output Function

In [9]:
def Output(a, b): 
    res = {**a, **b} # Merging the outputs from both the reducers
    return res 

#### Multi-threading Function - processor executes multiple threads concurrently.

In [10]:
import threading
import queue
def threading_func(input1,input2,function):
    queue1=queue.Queue()
    queue2=queue.Queue()
    t1 = threading.Thread(target=function, args=(input1,queue1)) 
    t2 = threading.Thread(target=function, args=(input2,queue2)) 
  
    # starting thread 1 
    t1.start() 
    # starting thread 2 
    t2.start() 
  
    t1.join() # wait until thread 1 is completely executed 
    t2.join()  # wait until thread 2 is completely executed 
    output_1=queue1.get()
    output_2=queue2.get() 
    return  output_1,output_2
    

#### Calling the functions

In [11]:
pride_cleaned=cleaning(pride)
split_files=split_function(pride_cleaned)
mapped_file=threading_func(split_files[0],split_files[1],mapper)
sorted_file=sorting(mapped_file[0],mapped_file[1])
partitions=partition(sorted_file)
reducedfile=threading_func(partitions[0],partitions[1],reducer)
combined_file=Output(reducedfile[0],reducedfile[1])

#### Final Output 

In [12]:
final_output=pd.DataFrame(combined_file.items(), columns=['Word', 'Frequency']) # converting to dataframe 
final_output

,Word,Frequency
0,a,1954
1,abatement,1
2,abhorrence,6
3,abhorrent,1
4,abide,1
5,abiding,1
6,abilities,6
7,able,54
8,ablution,1
9,abode,8


#### Exporting to csv file

In [13]:
final_output.to_csv("output1.csv",index=False, header=["word","Frequency"])